Start!

In [1]:
%reload_ext autoreload
%autoreload 2

In [17]:
import os
import shutil
import pandas as pd

from qiskit_metal import Dict
# import time
# import astropy.units as u
# import numpy as np
# import pandas as pd
# import astropy.constants as c
# #This imports the necessary path to draw the single pad transmon design. 
# import sys
# sys.path.append('Customized_Components')
# from rounded_single_pad import Round_TransmonPocket_Single as transmon
# import Transmon_property as trans_p
# import Transmon_specifications as jj
# from dolan_junction import DolanJunction as junction
# import Default_params as dp
# from pocket import TransmonPocket as pocket

# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
# from qiskit_metal import MetalGUI, Dict
# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

In [3]:
import make_sim_design as md

In [10]:
design = md.design
gui = md.gui

In [7]:
def collect_all_names(design):
    names = []
    for components in design.components:
        names.append(components)
    return names

In [25]:
def coupling_names(pos1, pos2):
    components = []
    items = collect_all_names(design)
    for component in items:
        if (pos1 in component) or (pos2 in component):
            if 'air' in component:
                pass
            else:
                components.append(component)
        if ('qubit(0' in component) and ((pos1 in component) or (pos2 in component)):
            components.append('wb_left'+component[-4])
        elif ('qubit(9' in component) and ((pos1 in component) or (pos2 in component)):
            components.append('wb_right'+component[-4])

    return components

In [11]:
from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.analyses.quantization import EPRanalysis
renderer_hfss = design.renderers.hfss
renderer_q3d = design.renderers.q3d


In [ ]:
original = r'C:\Users\slab\Desktop\Wendy-qiskit-code\ansys.png'
path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\Ansys_screenshoots\Simulation_0404'


nmode = 4

In [31]:
c1 = LOManalysis(design, "q3d")
c1.sim.setup.min_passes  = 10
c1.sim.setup.max_passes = 20
c1.sim.setup.freq_ghz = 5

c1.sim.renderer.options['wb_threshold'] ='72um'
c1.sim.renderer.options['x_buffer_width_mm'] = 0.5
c1.sim.renderer.options['y_buffer_width_mm'] = 0.5

'7nH'

In [12]:
eig_all = EPRanalysis(design, 'hfss')

hfss = eig_all.sim.renderer
eig_all.sim.renderer.options['wb_threshold'] = '72um'

eig_all.sim.setup.max_passes = 30
eig_all.sim.setup.max_delta_f = 0.1
eig_all.sim.setup.n_modes = nmode
eig_all.sim.renderer.options['x_buffer_width_mm'] = 0.5
eig_all.sim.renderer.options['y_buffer_width_mm'] = 0.5

In [14]:
eig_all.sim.renderer.options['wb_height'] = 0.2

In [5]:
first_rows = md.make_first_row(eig_all,rebuild = True)

NameError: name 'eig_all' is not defined

In [ ]:
# path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\Ansys_screenshoots\Simulation_0314\loop'
datas = md.pd.DataFrame()


md.sys.stdout = open(path+'output.txt','wt')


for p1,p2 in p:
    dat = {}
    dat['pos1'] = p1
    dat['pos2'] = p2

    # q1 = design.components['qubit'+p1]
    # q2 = design.components['qubit' + p2]


    # eig_all.sim.renderer.options['Lj'+p1] = q1.options['hfss_inductance']
    # eig_all.sim.renderer.options['Cj'+p1] = q1.options['hfss_capacitance']
    # eig_all.sim.renderer.options['Lj'+p2] = q2.options['hfss_inductance']
    # eig_all.sim.renderer.options['Cj'+p2] = q2.options['hfss_capacitance']

    # eig_all.sim.setup.vars = {}
    # eig_all.sim.setup.vars['Lj'+p1] = q1.options['hfss_inductance']
    # eig_all.sim.setup.vars['Cj'+p1] = q1.options['hfss_capacitance']
    # eig_all.sim.setup.vars['Lj'+p2] = q2.options['hfss_inductance']
    # eig_all.sim.setup.vars['Cj'+p2] = q2.options['hfss_capacitance']
    # eig_all.sim.setup.vars
    
    # = {l_name:Lj, c_name:Cj}
    

    components = coupling_names(p1, p2)

    c1.sim.run(components=components)#, open_terminations=[('Q1', 'a')])#, ('Q1', 'bus1'), ('Q1', 'bus2')])
    c1.sim.capacitance_matrix 
    
    c1.sim._get_results_from_renderer()
    c_mat = c1.sim.capacitance_matrix
    renderer_q3d.clean_active_design()
    c1  = c_mat['a_connector_pad_Q1']['pad_top_Q1']
    c2  = c_mat['a_connector_pad_Q1']['pad_top_Q2']
    c12  = c_mat['pad_top_Q1']['pad_top_Q2']

    dat['C_Q1_Q2(fF)'] = c12
    dat['C_Q1_g(fF)'] = c1
    dat['C_Q2_g(fF)'] = c2
    
    count = 0
    while count<4:
        try:
            eig_all.sim.run(name="all_freqs", components=components)#, open_terminations=[('Q1', 'a')])
        except:
            print('count')
            renderer_hfss.clean_active_design()
            count += 1
            if count>=3:
                dat['Freq'+str(i+1)+'(GHz)'] = 0
                for i in range(nmode):
                    dat['Freq_EPR'+str(i+1)+'(MHz)'] = 0
                    for j in range(i+1):
                        dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = 0
                break
        else:
            convergence = pd.read_csv('hfss_eig_f_convergence.csv')
            conv = convergence.dropna()
        
            ind = list(conv.keys())[1:]
            for i in range(nmode):
                freq = conv[ind[(i)]].values[-1]
                dat['Freq'+str(i+1)+'(GHz)'] = freq
                
            for i in range(nmode):
                    hfss.modeler._modeler.ShowWindow()
                    hfss.set_mode(i+1,'Setup')
                    hfss.plot_ansys_fields('main')
                    hfss.save_screenshot()
                    hfss.clear_fields(['main'])
                    
                    actual = r'C:\Users\slab\Desktop\Wendy-qiskit-code\f1{:.3f}_f2{:.3f}_1mm.png'.format(p1,)
                    os.rename(original, actual)
                    fail = True
                    i = 0
                    while fail:

                        try:
                            dest = shutil.move(actual, path)
                        except:
                            i+= 1
                            os.rename(actual,(actual+'{}'.format(i)))
                            actual = (actual+'{}'.format(i))
                        else:
                            fail = False
                    
            #Run the EPR analysis
            try:
                del eig_all.setup.junctions['jj']
            except:
                print('saddd')
            eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                            Lj_variable='Lj1', Cj_variable='Cj1')
            eig_all.setup.sweep_variable = 'Lj1'
            eig_all.run_epr()

            #Save the EPR data
            chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
            freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]
            
            renderer_hfss.clean_active_design()
            for i in range(nmode):
                freq = freq_EPR[i]
                dat['Freq_EPR'+str(i+1)+'(MHz)'] = freq
                for j in range(i+1):
                    dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = chi[i][j]

            data = pd.DataFrame(dat, index = [0])
            datas = pd.concat([datas,data], ignore_index=True)
            break


    datas.to_csv('data\two_qubit_1mm_0427.csv')


In [8]:
collect_all_names(design)

['wb_left0',
 'wb_left1',
 'wb_left2',
 'wb_left3',
 'wb_left4',
 'wb_left5',
 'wb_left6',
 'wb_left7',
 'wb_left8',
 'wb_left9',
 'wb_right0',
 'wb_right1',
 'wb_right2',
 'wb_right3',
 'wb_right4',
 'wb_right5',
 'wb_right6',
 'wb_right7',
 'wb_right8',
 'wb_right9',
 'qubit(0,0)',
 'jj(0,0)',
 'TQ(0,0)',
 'cpw_(0,0)',
 'airbridges(0,0)',
 'qubit(1,0)',
 'jj(1,0)',
 'TQ(1,0)',
 'cpw_(1,0)',
 'airbridges(1,0)',
 '(1,0)CPW(0,0)',
 'airbridge_connects(1,0)(0,0)',
 'qubit(2,0)',
 'jj(2,0)',
 'TQ(2,0)',
 'cpw_(2,0)',
 'airbridges(2,0)',
 '(2,0)CPW(1,0)',
 'airbridge_connects(2,0)(1,0)',
 'qubit(3,0)',
 'jj(3,0)',
 'TQ(3,0)',
 'cpw_(3,0)',
 'airbridges(3,0)',
 '(3,0)CPW(2,0)',
 'airbridge_connects(3,0)(2,0)',
 'qubit(4,0)',
 'jj(4,0)',
 'TQ(4,0)',
 'cpw_(4,0)',
 'airbridges(4,0)',
 '(4,0)CPW(3,0)',
 'airbridge_connects(4,0)(3,0)',
 'qubit(5,0)',
 'jj(5,0)',
 'TQ(5,0)',
 'cpw_(5,0)',
 'airbridges(5,0)',
 '(5,0)CPW(4,0)',
 'airbridge_connects(5,0)(4,0)',
 'qubit(6,0)',
 'jj(6,0)',
 'TQ(6,0)'

In [11]:
'qubit(3,0)'[-4]

'3'

In [4]:
design.components

{1: name:    wb_left0
class:   LaunchpadWirebond     
options: 
  'pos_x'             : '-3.4mm',                     
  'pos_y'             : '4.8mm',                      
  'orientation'       : 270,                          
  'chip'              : 'main',                       
  'layer'             : '5',                          
  'trace_width'       : '12.4 um',                    
  'trace_gap'         : '6.99 um',                    
  'lead_length'       : '8um',                        
  'pad_width'         : '200um',                      
  'pad_height'        : '100um',                      
  'pad_gap'           : '50um',                       
  'taper_height'      : '50um',                       
module:  qiskit_metal.qlibrary.terminations.launchpad_wb
id:      1
, 2: name:    wb_left1
class:   LaunchpadWirebond     
options: 
  'pos_x'             : '-4.8mm',                     
  'pos_y'             : '3.4mm',                      
  'orientation'       : 0,       

# Export

In [5]:
# design.chips.main.size['size_x'] = '10mm'
# design.chips.main.size['size_y'] = '10mm'
# gui.rebuild()
a_gds = design.renderers.gds

NameError: name 'design' is not defined

In [ ]:
a_gds.options.cheese.edge_nocheese = '100um'
a_gds.options.no_cheese.buffer = '50um'
a_gds.options.cheese.cheese_1_radius = '70um'

In [ ]:
qubit_layer = 5
junction_layer = 2
ab_layer = 8
ab_square_layer = 9

In [ ]:
a_gds.options

In [ ]:
a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = True
a_gds.options['cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = True
a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
a_gds.options['cheese']['view_in_file']['main'][1] = True
a_gds.options['no_cheese']['view_in_file']['main'][1] = True

In [ ]:
a_gds.options

In [ ]:
a_gds.export_to_gds('export_test_2.gds')